In [1]:
import pandas as pd
import numpy as np
import sys
from pandas import Series, DataFrame
from datetime import datetime
from calendar import monthrange
sys.path.append("../../")
from utils import model_utils
from utils import db_utils
from utils import feature_utils
from utils import iefp_data_utils
from utils import pandas_utils
from sklearn import preprocessing

import math
from dateutil.relativedelta import relativedelta

## for testing - don't include in final script
import datetime

def timestamp():
    global x
    y=x
    x=datetime.datetime.now()
    return str(datetime.datetime.now()-y)

from utils import model_utils

/opt/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from calendar import monthrange
from datetime import datetime, timedelta

def monthdelta(d1, d2):
    delta = 0
    while True:
        mdays = monthrange(d1.year, d1.month)[1]
        d1 += timedelta(days=mdays)
        if d1 <= d2:
            delta += 1
        else:
            break
    return delta

In [3]:
conn = db_utils.connect_to_db()
apps,movs = iefp_data_utils.get_clean_data(conn)
labelled_apps = feature_utils.get_ltu_label(apps,movs)

Generating LTU/Non-LTU labels


In [4]:
#Demographic feature functions
def apps_cancelled_within_n_months(x,n=12):
    return x[(x['movement_type'].isin(['cancellation'])) & (x['months_after_app'] < n)]['application_id'].unique()

def apps_placed_within_n_months(x,n=12):
    return x[(x['movement_result'].isin(['ADMITIDO / COLOCADO'])) & (x['months_after_app'] < n)]['application_id'].unique()

#Add column to show how many months after the application the movement occurred
def monthdelta(d1, d2):
    delta = 0
    while True:
        mdays = monthrange(d1.year, d1.month)[1]
        d1 += timedelta(days=mdays)
        if d1 <= d2:
            delta += 1
        else:
            break
    return delta

def difftime_in_months(timeA,timeB):
    return (timeA-timeB)/np.timedelta64(1, 'M')


def get_ltu_label(apps,movs):
    app_date_dict = dict(zip(apps["table_index"],apps["app_start_date"]))
    movs["app_start_date"] = movs["application_id"].map(app_date_dict)
    movs["months_after_app"] = difftime_in_months(movs["movement_event_date"],movs["app_start_date"])
    
    print "Generating LTU/Non-LTU labels"
    cancelled_12mo = apps_cancelled_within_n_months(movs)
    placed_12mo = apps_placed_within_n_months(movs)
    
    non_ltu_apps = np.unique(np.append(cancelled_12mo,placed_12mo))
    apps['ltu'] = np.logical_not(apps['table_index'].isin(non_ltu_apps))
    last_data_date = max(apps['app_start_date'])
    apps.loc[difftime_in_months(last_data_date,apps['app_start_date']) < 12,'ltu'] = False
    
    return apps

def findSeason(x):
    month=x.month
    if month in [12,1,2]:
        return "Winter"
    if month in [3,4,5]:
        return "Spring"
    if month in [6,7,8]:
        return "Summer"
    if month in [9,10,11]:
        return "Autumn"

def isPortuguese(x):
    if x=="PT":
        return "PT"
    else:
        return "OT"

def lookingForFirstJob(x):
    if "NOVO" in x:
        return "first_job"
    else:
        return "not_first_job"

def createEducationBuckets(x):
    HS=["12"]
    #SL means  able to write, but no school degree,NS means not able to write
    NR=["NS"]
    U6=["SL","04","06"]
    U11=["09","11"]
    
    #The rest is different kinds of higher degree
    if x in HS:
        return "HS"
    if x in NR:
        return "NR"
    if x in U6:
        return "U6"
    if x in U11:
        return "U11"
    else:
        return "MHS"

def civilStatus(x):
    if x=="S":
        return "S"
    if x=="C":
        return "M"
    else :
        return "O"

def ageBucket(x):
    if x<30:
        return "age<30"
    elif x<50:
        return "30<age<50"
    else:
        return "age>50"

def findMainBucket(profession):
    profDict={0:"Miliatary",1:"Managment",2:"Higer level professionals",3:"Lower level professionals",4:"Office jobs",5:"Sales",
              6:"Fising and farming",7:"Craft and trade",8:"Factory work",9:"Elementary"}
    if pd.isnull(profession):
        return "Unknown"
    else:
        professionInt=int(str(profession)[0:1])
        return profDict[professionInt]
    
def findMainEducation(education):
    eduDict={0:"general",1:"Education",2:"ArtsAndHumanities",3:"SocialScienceTradeAndLaw",4:"STEM",5:"EngineeringAndConstruction",6:"Agriculture",7:"HealthAndSocialProtection",8:"Service",9:"Unkown"}
    if pd.isnull(education):
        return eduDict[9]
    else:
        professionInt=int(str(education)[0:1])
        return eduDict[professionInt]

def preferredJobRegion(letter):
    if letter == "C":
        return "NearJobCenter"
    elif letter == "R":
        return "GreaterLisbonArea"
    elif letter == "N":
        return "Available"
    elif pd.isnull(letter):
        return "Unknown"
    else:
        return "Error"

def employmentPlan(letter):
    if letter == "S":
        return "exists"
    elif letter =="N":
        return "no plan"
    else:
        return "Error"
    
def applicationOrigin(origin):
    if origin == "SIGAE":
        return "SIGAE"
    elif origin == "LSE":
        return "LSE"
    elif pd.isnull(origin):
        return "unknown"
    else:
        return "Error"
    
def intendedRegime(regime):
    if regime == "COMPLETO":
        return "Full-time"
    elif regime == "PARCIAL":
        return "Part-time"
    else:
        return "Error"

def dependentsBucket(n):
    if n == 0:
        return "0"
    elif n == 1:
        return "1"
    elif n == 2:
        return "2"
    elif n == 3:
        return "3"
    elif pd.isnull(n):
        return "Unknown"
    else:
        return "4+"
    
def experienceBuckets(n):
    if pd.isnull(n):
        return "Unknown"
    elif n == 0:
        return "0"
    elif n<24:
        return "<2yr"
    elif n<60:
        return "<5yr"
    elif n<120:
        return "<10yr"
    elif n<240:
        return "<20yr"
    else:
        return "20+yr"


In [5]:
#Variables definitions
feature_set_name = 'dynamic'
train_st_date = pd.to_datetime('2015-04-30')
test_st_date = pd.to_datetime('2016-04-30')
test_window_size = pd.Timedelta('365D')
ltu_length = 12

In [19]:
#Functions definitions
def split_train_test_apps(apps_df,movs_df,test_date,train_start_date,ltu_length):
    selected_apps = model_utils.filter_by_time_range(apps_df,'app_start_date',train_start_date,test_date).sort_values('app_start_date')
    selected_movs = model_utils.filter_by_time_range(movs_df,'movement_event_date',train_start_date,test_date).sort_values('movement_event_date')
    exitted = model_utils.apps_exited_before_date(selected_movs,test_date)
    test_apps = selected_apps[(np.logical_not(selected_apps['table_index'].isin(exitted))) &
                              (pandas_utils.difftime_in_months(test_date,selected_apps['app_start_date']) < ltu_length)]
    
    train_apps = selected_apps[(np.logical_not(selected_apps['table_index'].isin(test_apps['table_index'])))]
    
    train_movs = selected_movs[(np.logical_not(selected_movs['application_id'].isin(test_apps['table_index'])))]
    return train_apps, test_apps, train_movs

def get_cancellation_date(x):
    return x[x['movement_type'].isin(['cancellation'])].sort_values('movement_event_date').groupby(['application_id']).first().reset_index()[['application_id','movement_event_date']].rename(columns={'movement_event_date': 'cancellation_date'})

def get_placement_date(x):
    return x[x['movement_result'].isin(['ADMITIDO / COLOCADO'])].sort_values('movement_event_date').groupby(['application_id']).first().reset_index()[['application_id','movement_event_date']].rename(columns={'movement_event_date': 'placement_date'})

def get_last_active_date(apps,movs,ref_date):
    apps_cancellations = get_cancellation_date(movs)
    apps_placements = get_placement_date(movs)
    apps_length = apps.merge(apps_cancellations,how='left', left_on='table_index',right_on='application_id').merge(apps_placements,how='left', left_on='table_index',right_on='application_id')[['table_index','app_start_date','cancellation_date','placement_date']]
    apps_length['ref_date'] = ref_date
    apps_length['last_active_date'] = apps_length[['cancellation_date', 'placement_date','ref_date']].min(axis=1)
    
    return apps_length[['table_index','app_start_date','last_active_date']]    

def get_app_length(apps,movs,ref_date=None):
    apps_last_active_date = get_last_active_date(train_apps,movs,ref_date)
    apps_last_active_date['app_length'] = apps_last_active_date['last_active_date'] - apps_last_active_date['app_start_date']
    
    return apps_last_active_date

def extend_data(apps,movs,labels,ref_date,time_delta):
    apps_length = get_app_length(apps,movs,ref_date)
    apps_length = apps_length.merge(labels,on='table_index')
    toDataFrame=[]
    for i in xrange(0,apps_length.shape[0]):
        app_id = apps_length['table_index'][i]
        app_st_date = apps_length['app_start_date'][i]
        app_length = apps_length['app_length'][i]
        app_label = apps_length['ltu'][i]
        for j in xrange(0,int(math.ceil(app_length/pd.Timedelta('30D')))+1):
            toDataFrame.append([app_id,app_st_date,app_st_date + (j*relativedelta(months=1)),app_label])
    extended_data = pd.DataFrame(toDataFrame,columns=['application_id','app_start_date','ref_date','ltu'])
    return extended_data

def apps_cancelled_within_n_months(x,n=12):
    return x[(x['movement_type'].isin(['cancellation'])) & (x['months_after_app'] < n)]['application_id'].unique()

def apps_placed_within_n_months(x,n=12):
    return x[(x['movement_result'].isin(['ADMITIDO / COLOCADO'])) & (x['months_after_app'] < n)]['application_id'].unique()

def difftime_in_months(timeA,timeB):
    return (timeA-timeB)/np.timedelta64(1, 'M')

def get_ltu_label(apps,movs):
    app_date_dict = dict(zip(apps["table_index"],apps["app_start_date"]))
    movs["app_start_date"] = movs["application_id"].map(app_date_dict)
    movs["months_after_app"] = difftime_in_months(movs["movement_event_date"],movs["app_start_date"])
    
    print "Generating LTU/Non-LTU labels"
    cancelled_12mo = apps_cancelled_within_n_months(movs)
    placed_12mo = apps_placed_within_n_months(movs)
    
    non_ltu_apps = np.unique(np.append(cancelled_12mo,placed_12mo))
    apps['ltu'] = np.logical_not(apps['table_index'].isin(non_ltu_apps))
    last_data_date = max(apps['app_start_date'])
    apps.loc[difftime_in_months(last_data_date,apps['app_start_date']) < 12,'ltu'] = False
    
    return apps

def get_ltu_label_on_date(apps,movs,date,ltu_length):
    apps_movs = pandas_utils.filter_by_time_range(movs[movs['application_id'].isin(apps['table_index'])],'movement_event_date',None,date)
    app_date_dict = dict(zip(apps["table_index"],apps["app_start_date"]))
    apps_movs["app_start_date"] = apps_movs["application_id"].map(app_date_dict)
    apps_movs["months_after_app"] = difftime_in_months(apps_movs["movement_event_date"],apps_movs["app_start_date"])
    
    print "Generating LTU/Non-LTU labels"
    cancelled_12mo = apps_cancelled_within_n_months(apps_movs,ltu_length)
    placed_12mo = apps_placed_within_n_months(apps_movs,ltu_length)
    
    non_ltu_apps = np.unique(np.append(cancelled_12mo,placed_12mo))
    apps['ltu'] = np.logical_not(apps['table_index'].isin(non_ltu_apps))
    apps.loc[difftime_in_months(date,apps['app_start_date']) < ltu_length,'ltu'] = False
    
    return apps[['table_index','ltu']]

In [20]:
#Splitting train/test datasets
train_apps,test_apps,train_movs = split_train_test_apps(apps,movs,test_st_date,train_st_date,ltu_length)

In [21]:
#Generate LTU Label
train_labels = get_ltu_label_on_date(train_apps,movs,test_st_date,ltu_length)
test_labels = get_ltu_label_on_date(test_apps,movs,test_st_date + test_window_size,ltu_length)

Generating LTU/Non-LTU labels
Generating LTU/Non-LTU labels


In [22]:
#Extend data
train_extended = extend_data(train_apps,movs,train_labels,test_st_date,pd.Timedelta('30D'))

In [23]:
def add_demographics(feature_matrix, apps):
    #Create DF with the demographic features per application_id
    demo_df = apps[["table_index"]]
    
    demo_df["age"] = apps["ute_idade"] #redo age variable to make dyanamic?
    demo_df["gender"] = apps["sexo"]
    demo_df["is_re_registriation"] = apps["candidatura_rinsc"]
    demo_df["soc_ben"] = apps["sub_rsi"]
    demo_df["age2"]=demo_df["age"]**2
    demo_df["age"]=preprocessing.scale(demo_df["age"].astype(float),copy=False)
    demo_df["age2"]=preprocessing.scale(demo_df["age2"].astype(float),copy=False)
    demo_df["season"]=apps["app_start_date"].apply(findSeason)
    demo_df["nationality"]=apps["cnacionalidade"].apply(isPortuguese)
    demo_df["education"]=apps["chabilitacao_escolar"].apply(createEducationBuckets) 
    demo_df["first_job"]=apps["dcategoria"].apply(lookingForFirstJob)
    demo_df["time_since_exit"]=(pd.to_datetime(apps["candidatura_data"])-pd.to_datetime(apps["reinscricao_ult_saida_data"])).dt.days
    demo_df["time_since_exit"].fillna(500,inplace=True)
    demo_df["time_since_exit"]=pd.to_numeric(demo_df["time_since_exit"])
    demo_df["time_since_exit"]=preprocessing.scale(demo_df["time_since_exit"], copy=False)
    demo_df["is_disabled"]=apps["cdeficiencia"].apply(lambda x: "N" if x==0 else "S")
    demo_df["civil_status"]=apps["ute_estado_civil"].apply(civilStatus)
    demo_df["has_course_area"]=apps["darea_curso_tabela_em_activo"]
    demo_df["has_course_area"].fillna(0,inplace=True)
    demo_df["has_course_area"]=demo_df["has_course_area"].apply(lambda x: "S" if x==0 else "N")
    demo_df["has_prof_area"]=apps["darea_formacao_tabela_em_activo"]
    demo_df["has_prof_area"].fillna(0,inplace=True)
    demo_df["has_prof_area"]=demo_df["has_prof_area"].apply(lambda x: "S" if x==0 else "N")
    demo_df["age_category"]=demo_df["age"].apply(ageBucket)
    demo_df["number_dependents"]=apps["ute_nr_pessoas_cargo"].astype(float).fillna(0)
    demo_df["number_dependents"]=preprocessing.scale(demo_df["number_dependents"], copy=False)
    demo_df["experience_intended_prof"]=apps["candidatura_prof_pret_tempo_pratica"].astype(float).fillna(0)
    demo_df["experience_intended_prof"]=preprocessing.scale(demo_df["experience_intended_prof"], copy=False)
    demo_df["experience_prev_prof"]=apps["sit_anterior_prof_tempo_pratica"].astype(float).fillna(0)
    demo_df["experience_prev_prof"]=preprocessing.scale(demo_df["experience_prev_prof"], copy=False) 
    demo_df["intended_prof_1"]=apps["cnp_pretendida"].apply(findMainBucket)
    demo_df["intended_prof_2"]=apps["cpp_pretendida"].apply(findMainBucket)
    demo_df["intended_prof"]=np.where(pd.isnull(demo_df["intended_prof_1"]),demo_df["intended_prof_2"],demo_df["intended_prof_1"])
    del demo_df["intended_prof_1"]
    del demo_df["intended_prof_2"]  
    demo_df["previous_prof_1"]=apps["cnp_anterior"].apply(findMainBucket)
    demo_df["previous_prof_2"]=apps["cpp_anterior"].apply(findMainBucket)
    demo_df["previous_prof"]=np.where(pd.isnull(demo_df["previous_prof_1"]),demo_df["previous_prof_2"],demo_df["previous_prof_1"])
    del demo_df["previous_prof_1"]
    del demo_df["previous_prof_2"]   
    demo_df["training_area"]= apps["carea_formacao_tabela_em_activo"].apply(findMainEducation)   
    #demo_df['app_start_date'] = pd.to_datetime(apps['app_start_date'])  #already in the extended data
    demo_df['preferred_location']=apps['candidatura_local_trabalho'].apply(preferredJobRegion)
    demo_df['app_year'] = apps['app_start_date'].apply(lambda x: x.year)
    demo_df['app_month'] = apps['app_start_date'].apply(lambda x: x.month)
    demo_df['employment_plan'] = apps['ute_plano_emprego'].apply(employmentPlan)
    demo_df['prev_employment_plan'] = apps['ute_plano_emprego_anterior'].apply(employmentPlan)
    demo_df['app_origin'] = apps['candidatura_origem'].apply(applicationOrigin)
    demo_df['intended_regime'] = apps['dregime_contrato_pretendido'].apply(intendedRegime)
    demo_df['dependents_bucket'] = apps['ute_nr_pessoas_cargo'].astype(float).apply(dependentsBucket)
    demo_df['exp_intended_prof_buckets'] = apps['candidatura_prof_pret_tempo_pratica'].astype(float).apply(experienceBuckets)
    demo_df['exp_previous_prof_buckets'] = apps['sit_anterior_prof_tempo_pratica'].astype(float).apply(experienceBuckets)
    
    #map to feature matrix using application ID
    feature_matrix = pd.merge(feature_matrix, demo_df, how='left', left_on='application_id', left_index=True, right_on='table_index')
    return feature_matrix

In [24]:
def generate_matrix(extended_data, apps_cropped, movs_cropped, feature_set_list):
    #map movements to application start date
    #app_date_dict = dict(zip(extended_data["table_index"],extended_data["app_start_date"]))
    #movs["app_start_date"] = movs["application_id"].map(app_date_dict)

    print "Generating matrix base" #generate basic matrix including appID, application date, months so far, and label
    feature_matrix = extended_data.copy()
    if "demographics" in feature_set_list:
        print "Adding demographic features"
        feature_matrix = add_demographics(feature_matrix, apps_cropped)
    
    if "dynamic" in feature_set_list:
        print "Adding dynamic features"
        feature_matrix = add_dynamic_features(feature_matrix, movs_cropped)
    
    #Removing Irrelevant features and create dummies for categorical variables
    irrelevant_features = ['application_id','app_start_date','ref_date']
    #feature_matrix.drop(irrelevant_features,1,inplace=True)
    #feature_matrix = pd.get_dummies(feature_matrix,drop_first=True)
    
    return feature_matrix

In [25]:
def add_dynamic_features(feature_matrix, movs):
    feature_matrix['time_so_far'] = feature_matrix['ref_date']-feature_matrix['app_start_date']
    feature_matrix['months_so_far'] = feature_matrix['time_so_far']/np.timedelta64(1,'M')
    
    for index, row in feature_matrix.iterrows():
        feature_matrix.loc[index,'moves_so_far']=len(movs[(movs['application_id']==row['application_id'])\
                                                          & (movs['movement_event_date']<row['ref_date'])])
#     for index in feature_matrix.index:
#         application_id = feature_matrix.loc[index,'application_id']
#         ref_date = feature_matrix.loc[index,'ref_date']
#         print movs[(movs['application_id']==application_id)]) #&\
         #(movs['movement_type']=="intervention") &\
         #(movs['movement_event_date']<ref_date)])
    #selected_movs = movs[movs['application_id'].isin(feature_matrix['application_id'])]
    #model_utils.filter_by_time_range(movs,'movement_event_date',None,test_date)
    #feature_matrix['months_so_far'] = feature_matrix['app_start_date'].apply(lambda x: monthdelta(x,ref_date))
    ##NEED TO FIX THESE
#     mov_counts_dict = dict(movs.groupby('application_id').movement_type.value_counts().unstack())
#     feature_matrix['interventions_so_far'] = feature_matrix['application_id'].apply(lambda x: mov_counts_dict['intervention'][x])
#     feature_matrix['interventions_so_far'].fillna(0,inplace=True)
#     feature_matrix['interviews_so_far'] =  feature_matrix['application_id'].apply(lambda x: mov_counts_dict['interview'][x])
#     feature_matrix['interviews_so_far'].fillna(0,inplace=True)
#     feature_matrix['convocations_so_far'] =  feature_matrix['application_id'].apply(lambda x: mov_counts_dict['convocation'][x])
#     feature_matrix['convocations_so_far'].fillna(0,inplace=True)
    #feature_matrix['apps_in_last_3_yrs'] need to take in previous app data
    return feature_matrix

In [45]:
pd.options.mode.chained_assignment = None
train_matrix = generate_matrix(train_extended,train_apps,train_movs,["dynamic"])

Generating matrix base
Adding dynamic features


KeyboardInterrupt: 

In [49]:
train_matrix[['application_id','ref_date','moves_so_far']].sort_values(['application_id','ref_date'])

,application_id,ref_date,moves_so_far
82893,82894,2015-08-06,9.0
82893,82894,2015-09-06,9.0
82893,82894,2015-10-06,9.0
83240,83241,2015-09-24,6.0
83240,83241,2015-10-24,6.0
84834,84835,2015-07-23,35.0
84834,84835,2015-08-23,35.0
84834,84835,2015-09-23,35.0
84834,84835,2015-10-23,35.0
84834,84835,2015-11-23,35.0


In [73]:
train_movs['movement_event_date'][679141]<train_matrix['ref_date'].iloc[0]

False

In [72]:
train_matrix['ref_date'].iloc[0]

Timestamp('2015-04-30 00:00:00')

In [58]:
train_movs.head()

,ute_id,movement_event_date,application_id,movement_type,movement_subtype,movement_result,movement_index,app_start_date,months_after_app
679141,3917598,2015-04-30,99624,application,DESEMPREGADO-NOVO EMPREGO,,99624,2015-04-30,0.000000
606593,3697365,2015-04-30,92918,convocation,CONVOCATÃRIA GIP,NÃO COMPARECEU INJUSTIFICADAMENTE,200980,2014-01-29,14.981827
606592,3697365,2015-04-30,92918,convocation,CONVOCATÃRIA GIP,NÃO COMPARECEU INJUSTIFICADAMENTE,200979,2014-01-29,14.981827
43241,367021,2015-04-30,92906,interview,588551128,RECUSA DA ENTIDADE EMPREGADORA - NÃO MARCAÃÃ...,127692,2014-10-08,6.702396
825828,4347531,2015-04-30,86722,category_change,DESEMPREGADO-NOVO EMPREGO,DESEMP NOVO EMPREGO - INDISPONÃVEL,92090,2014-04-11,12.616275


In [74]:
len(train_movs[(train_movs['application_id']==84835)&(train_movs['movement_event_date']<train_matrix['ref_date'].iloc[0])])

0

In [ ]:
pd.merge(left = train_extended, train_movs)

In [34]:
extended_dates = train_extended[['application_id','ref_date']].rename(columns={'ref_date':'movement_event_date'})
extended_dates['movement_type'] = 'ref_date'
mov_dates = train_movs[['application_id','movement_type','movement_event_date']]
extended_movs = pd.concat([extended_dates,mov_dates]).sort_values(by=['application_id','movement_event_date'])

In [37]:
interview_count = 0
convocation_count = 0
intervention_count = 0
previous_id = -1
for index,row in extended_movs.iterrows():
    if row['application_id']!=previous_id:
        interview_count = 0
        convocation_count = 0
        intervention_count = 0 
    if row['movement_type'] == "interview":
        interview_count +=1
    elif row['movement_type'] == "convocation":
        convocation_count +=1
    elif row['movement_type'] == "intervention":
        intervention_count +=1
    elif row['movement_type'] == "ref_date":
        row['interviews_so_far'] = interview_count
        row['convocations_so_far'] = convocation_count
        row['interventions_so_far'] = intervention_count

In [40]:
len(extended_movs)

125231

In [ ]:
train_extended.head()

In [ ]:
train_matrix.head()

In [ ]:
mov_counts.head()

In [ ]:
mov_counts = train_movs[['application_id','movement_type','movement_event_date']].sort_values(by=['application_id',"movement_type","movement_event_date"])

In [ ]:
mov_counts[mov_counts['application_id']==86722]

In [ ]:
#old version for reference
def generate_matrix_old(clean_apps,clean_movs,feature_set_name,ref_date,feature_file_path=None):

    app_date_dict = dict(zip(clean_apps["table_index"],clean_apps["app_start_date"]))
    clean_movs["app_start_date"] = clean_movs["application_id"].map(app_date_dict)
    
    feature_matrix = clean_apps[["table_index","app_start_date","ltu","ute_idade","sexo","candidatura_rinsc","sub_rsi"]].drop_duplicates(subset='table_index')
    feature_matrix.rename(columns={"table_index":"application_id", "ute_idade": "age", "sexo":"gender",\
                              "candidatura_rinsc":"is_re_registriation","sub_rsi":"soc_ben"}, inplace=True)
    
    print "Building model feature matrix"
    feature_matrix["age2"]=feature_matrix["age"]**2
    feature_matrix["age"]=preprocessing.scale(feature_matrix["age"].astype(float),copy=False)
    feature_matrix["age2"]=preprocessing.scale(feature_matrix["age2"].astype(float),copy=False)
    feature_matrix["season"]=clean_apps["app_start_date"].apply(findSeason)
    feature_matrix["nationality"]=clean_apps["cnacionalidade"].apply(isPortuguese)
    feature_matrix["education"]=clean_apps["chabilitacao_escolar"].apply(createEducationBuckets) 
    feature_matrix["first_job"]=clean_apps["dcategoria"].apply(lookingForFirstJob)
    feature_matrix["time_since_exit"]=(pd.to_datetime(clean_apps["candidatura_data"])-pd.to_datetime(clean_apps["reinscricao_ult_saida_data"])).dt.days
    feature_matrix["time_since_exit"].fillna(500,inplace=True)
    feature_matrix["time_since_exit"]=pd.to_numeric(feature_matrix["time_since_exit"])
    feature_matrix["time_since_exit"]=preprocessing.scale(feature_matrix["time_since_exit"], copy=False)
    feature_matrix["is_disabled"]=clean_apps["cdeficiencia"].apply(lambda x: "N" if x==0 else "S")
    feature_matrix["civil_status"]=clean_apps["ute_estado_civil"].apply(civilStatus)
    feature_matrix["has_course_area"]=clean_apps["darea_curso_tabela_em_activo"]
    feature_matrix["has_course_area"].fillna(0,inplace=True)
    feature_matrix["has_course_area"]=feature_matrix["has_course_area"].apply(lambda x: "S" if x==0 else "N")
    feature_matrix["has_prof_area"]=clean_apps["darea_formacao_tabela_em_activo"]
    feature_matrix["has_prof_area"].fillna(0,inplace=True)
    feature_matrix["has_prof_area"]=feature_matrix["has_prof_area"].apply(lambda x: "S" if x==0 else "N")
    feature_matrix["age_category"]=feature_matrix["age"].apply(ageBucket)
    feature_matrix["number_dependents"]=clean_apps["ute_nr_pessoas_cargo"].astype(float).fillna(0)
    feature_matrix["number_dependents"]=preprocessing.scale(feature_matrix["number_dependents"], copy=False)
    feature_matrix["experience_intended_prof"]=clean_apps["candidatura_prof_pret_tempo_pratica"].astype(float).fillna(0)
    feature_matrix["experience_intended_prof"]=preprocessing.scale(feature_matrix["experience_intended_prof"], copy=False)
    feature_matrix["experience_prev_prof"]=clean_apps["sit_anterior_prof_tempo_pratica"].astype(float).fillna(0)
    feature_matrix["experience_prev_prof"]=preprocessing.scale(feature_matrix["experience_prev_prof"], copy=False) 
    feature_matrix["intended_prof_1"]=clean_apps["cnp_pretendida"].apply(findMainBucket)
    feature_matrix["intended_prof_2"]=clean_apps["cpp_pretendida"].apply(findMainBucket)
    feature_matrix["intended_prof"]=np.where(pd.isnull(feature_matrix["intended_prof_1"]),feature_matrix["intended_prof_2"],feature_matrix["intended_prof_1"])
    del feature_matrix["intended_prof_1"]
    del feature_matrix["intended_prof_2"]  
    feature_matrix["previous_prof_1"]=clean_apps["cnp_anterior"].apply(findMainBucket)
    feature_matrix["previous_prof_2"]=clean_apps["cpp_anterior"].apply(findMainBucket)
    feature_matrix["previous_prof"]=np.where(pd.isnull(feature_matrix["previous_prof_1"]),feature_matrix["previous_prof_2"],feature_matrix["previous_prof_1"])
    del feature_matrix["previous_prof_1"]
    del feature_matrix["previous_prof_2"]   
    feature_matrix["training_area"]= clean_apps["carea_formacao_tabela_em_activo"].apply(findMainEducation)   
    feature_matrix['app_start_date'] = pd.to_datetime(feature_matrix['app_start_date'])
    
    feature_matrix['preferred_location']=clean_apps['candidatura_local_trabalho'].apply(preferredJobRegion)
    feature_matrix['app_year'] = clean_apps.app_start_date.apply(lambda x: x.year)
    feature_matrix['app_month'] = clean_apps.app_start_date.apply(lambda x: x.month)
    feature_matrix['employment_plan'] = clean_apps.ute_plano_emprego.apply(employmentPlan)
    feature_matrix['prev_employment_plan'] = clean_apps.ute_plano_emprego_anterior.apply(employmentPlan)
    feature_matrix['app_origin'] = clean_apps.candidatura_origem.apply(applicationOrigin)
    feature_matrix['intended_regime'] = clean_apps.dregime_contrato_pretendido.apply(intendedRegime)
    feature_matrix['dependents_bucket'] = clean_apps.ute_nr_pessoas_cargo.astype(float).apply(dependentsBucket)
    feature_matrix['exp_intended_prof_buckets'] = clean_apps.candidatura_prof_pret_tempo_pratica.astype(float).apply(experienceBuckets)
    feature_matrix['exp_previous_prof_buckets'] = clean_apps.sit_anterior_prof_tempo_pratica.astype(float).apply(experienceBuckets)
    
    if feature_set_name=="dynamic":
        feature_matrix['time_since_app'] = feature_matrix['app_start_date'].apply(lambda x: monthdelta(x,ref_date))
        #feature_matrix['apps_in_last_3_yrs'] need to take in previous app data
        mov_counts_dict = dict(clean_movs.groupby('application_id').movement_type.value_counts().unstack())
        feature_matrix['interventions_so_far'] = feature_matrix['application_id'].apply(lambda x: mov_counts_dict['intervention'][x])
        feature_matrix['interventions_so_far'].fillna(0,inplace=True)
        feature_matrix['interviews_so_far'] =  feature_matrix['application_id'].apply(lambda x: mov_counts_dict['interview'][x])
        feature_matrix['interviews_so_far'].fillna(0,inplace=True)
        feature_matrix['convocations_so_far'] =  feature_matrix['application_id'].apply(lambda x: mov_counts_dict['convocation'][x])
        feature_matrix['convocations_so_far'].fillna(0,inplace=True) 


    #Removing Irrelevant features and create dummies for categorical variables
    irrelevant_features = ['app_start_date']
    feature_matrix.drop(irrelevant_features,1,inplace=True)
    feature_matrix = pd.get_dummies(feature_matrix,drop_first=True)
   
    if feature_file_path!=None: 
        print "Writing model matrix to file"
        feature_matrix.to_csv(matrix_file_path, index=False, encoding='utf8')
    return feature_matrix


In [ ]:
train_matrix = generate_matrix(train_apps,train_movs,feature_set_name,train_end_date)

In [ ]:
## getting time since app
%matplotlib inline
train_matrix.time_in_sys.value_counts().sort_index().plot(kind='bar')

In [ ]:
train_matrix.iloc[:,0:10].head()

In [ ]:
## Getting actions since app

In [ ]:
mov_counts = train_movs.groupby('application_id').movement_type.value_counts().unstack()

In [ ]:
mov_counts_dict = dict(mov_counts)

In [ ]:
mov_counts_dict['intervention'][86812]

In [ ]:
train_apps['interventions_so_far'] = train_apps['table_index'].\
apply(lambda x: mov_counts_dict['intervention'][x])
train_apps['interventions_so_far'].fillna(0,inplace=True)

In [ ]:
train_apps.interventions_so_far.value_counts()

In [ ]:
len(train_matrix)

In [ ]:
train_matrix.ltu.mean()

In [ ]:
train_apps.tail()

In [ ]:
## Getting info from previous apps

In [ ]:
train_prev_app_st = split_date - train_timedelta - test_timedelta - pd.Timedelta("1095D")
test_prev_app_st = split_date - test_timedelta - pd.Timedelta("1095D")

In [ ]:
train_prev_app_st

In [ ]:
train_prev_apps_movs = model_utils.filter_by_time_range(movs,'movement_event_date',\
                                 train_prev_app_st,train_end_date)
train_prev_apps_apps = model_utils.filter_by_time_range(apps,'app_start_date',\
                                 train_prev_app_st,train_end_date)

In [ ]:
train_prev_apps_apps.groupby('ute_id').table_index.nunique()

In [ ]:
movs.head()

In [ ]:
apps.ute_nr_pessoas_cargo.value_counts()

In [ ]:
def experienceBuckets(n):
    if pd.isnull(n):
        return "Unknown"
    elif n == 0:
        return "0"
    elif n<24:
        return "<2yr"
    elif n<60:
        return "<5yr"
    elif n<120:
        return "<10yr"
    elif n<240:
        return "<20yr"
    else:
        return "20+yr"
    
#apps.candidatura_prof_pret_tempo_pratica.astype(float).value_counts(dropna=False).sort_index()
apps.candidatura_prof_pret_tempo_pratica.astype(float).apply(experienceBuckets).value_counts()
apps.sit_anterior_prof_tempo_pratica.astype(float).apply(experienceBuckets).value_counts()

In [ ]:
%matplotlib inline
apps.sit_anterior_prof_tempo_pratica.astype(float).apply(lambda x: x/12).plot.hist(bins=200, xlim=(0,30))

In [ ]:
apps.sit_anterior_prof_tempo_pratica.value_counts().sort_values(ascending=False)

In [ ]:
print apps.cae_anterior.value_counts(dropna=False)

In [ ]:
apps.columns

In [ ]:
train_matrix.head()

In [ ]:
print train_matrix.ltu.value_counts(normalize=False)
print test_matrix.ltu.value_counts(normalize=False)

In [ ]:
for column in train_matrix.columns:
    print train_matrix[column].value_counts(dropna=False)

In [ ]:
train_matrix.columns[0]

In [ ]:
# scale to [0,1]?
# dependents and experience into buckets?
# what if categories don't appear in both train and test (like month)?

## Regex

In [ ]:
import re

In [ ]:
features_list = ['age', 'age2', 'year', \
            'month', 'dynamic_time_since_app', 'dynamic_last_interview']
feature_string = 'age age2 year month dynamic_time_since_app dynamic_last_int'

#re.match('age',features)